In [6]:
import json
import re # regex
import os # build fild paths that work on any OS

In [7]:
# --- Step 1: Read the File and Confirm Path ---

try:
    current_directory = os.getcwd()
    file_path = os.path.join(current_directory, '..', 'vocab', 'domislang.txt')

    print(f"Attempting to read from: {file_path}\n")

    # Use 'with open' to safely open the file.
    # Specify encoding='utf-8' to handle special characters correctly.
    with open(file_path, 'r', encoding='utf-8') as f:
        # Read only the first 500 characters to test readability.
        content_preview = f.read(500)

        print("--- File Preview (First 500 Characters) ---")
        print(content_preview)
        print("\n-------------------------------------------")
        print("\n✅ Success! File read correctly. We are ready for Step 2.")

except FileNotFoundError:
    print("❌ ERROR: File not found.")
    print(f"I looked for the file at this path: {file_path}")
    print("Please make sure your notebook is in the correct 'scripts' directory and the file exists.")

Attempting to read from: /Users/willi.wonkah/_Development/dominican-chatbot/json-conversion/scripts/../vocab/domislang.txt

--- File Preview (First 500 Characters) ---
﻿# A
1. Abimbao - This term describes someone who has just been beaten and has visible swelling as a result / beaten up.
EX: "Juan y Manuel pelearon, yo no sé por qué, pero Manuel ta abimbao."
GS: "Juan y Manuel pelearon, yo no sé por qué, pero Manuel está mal herido."
EN: "Juan and Manuel had a fight, I don't know why, but Manuel is beaten up."
SYN: Golpeado / Mal herido
ANT:-
Usage: 8 / 10


2. Abombao - Full (When you eat a lot) / Stuffed.
EX: "Me comí un pica pollo y ahora toy abombao."
GS: 

-------------------------------------------

✅ Success! File read correctly. We are ready for Step 2.
